In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [30]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Resizing  # Import Resizing directly

# Step 1: Load and preprocess CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize the pixel values between 0 and 1
X_train = X_train / 255.0
X_test = X_test / 255.0

# Step 2: Load the pre-trained MobileNetV2 model (without top layers)
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,  # Exclude fully connected layer for transfer learning
                                               weights='imagenet')  # Load imagenet weights for image classification

# Step 3: Freeze the base model (pre-trained model won't be updated during training)
base_model.trainable = False

# Step 4: Add custom layers on top of the base model
model = models.Sequential([
    Resizing(224, 224),  # Correct usage with two arguments: (target height, target width)
    base_model,  # Add the pre-trained MobileNetV2 model
    layers.GlobalAveragePooling2D(),  # Add pooling layer to reduce dimensions before the final Dense layer
    layers.Dense(128, activation='relu'),  # Optional fully connected layer
    layers.Dense(10, activation='softmax')  # Output layer for 10 classes (CIFAR-10)
])

# Step 5: Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Step 6: Train the model on the CIFAR-10 training data
model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test))

# Step 7: Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test)

test_acc


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 1413s 899ms/step - accuracy: 0.7234 - loss: 0.7970 - val_accuracy: 0.7996 - val_loss: 0.5755
313/313 ━━━━━━━━━━━━━━━━━━━━ 223s 711ms/step - accuracy: 0.7975 - loss: 0.5774


0.7996000051498413